In [23]:
import polars as pl

In [24]:
pl.Config.set_fmt_str_lengths(40)

polars.config.Config

In [25]:
playlist = (
    pl.read_json("spotify-data/Playlist1.json")
    .explode("playlists")
    .unnest("playlists")
    .explode("items")
    .unnest("items")
    .unnest("track")
    .filter(
        pl.col("episode").is_not_null().not_(),  # drop episodes
    )
    .drop("episode", "localTrack", "audiobook", "description", "numberOfFollowers")
)
print(playlist.shape)
playlist.head()

(1150, 7)


name,lastModifiedDate,trackName,artistName,albumName,trackUri,addedDate
str,str,str,str,str,str,str
"""BANOOK""","""2024-01-26""","""Jesus to a Child""","""George Michael""","""Older""","""spotify:track:2SzCxX6M6vDwdEwnHDiTaY""","""2023-07-18"""
"""BANOOK""","""2024-01-26""","""O Willow Waly""","""Ashley Serena""","""O Willow Waly""","""spotify:track:6lI5gSYb2glYqp6SSCdN82""","""2023-07-18"""
"""BANOOK""","""2024-01-26""","""My Confession""","""Josh Groban""","""Closer""","""spotify:track:1jOLSsbFpAQhqMX65cbu2p""","""2023-07-18"""
"""BANOOK""","""2024-01-26""","""A Different Corner""","""George Michael""","""Ladies And Gentlemen... The Best Of Geor…","""spotify:track:6khoye7kysa6cLGY0AO9dO""","""2023-07-18"""
"""BANOOK""","""2024-01-26""","""Cry Me a River""","""Justin Timberlake""","""Justified""","""spotify:track:7Lf7oSEVdzZqTA0kEDSlS5""","""2023-07-18"""


In [26]:
print("No. of distinct playlists in library:", playlist["name"].n_unique())
print("No. of distinct tracks in library:", playlist["trackUri"].n_unique())
print("No. of distinct artists in library:", playlist["artistName"].n_unique())

No. of distinct playlists in library: 21
No. of distinct tracks in library: 1073
No. of distinct artists in library: 629


In [27]:
# preprocess playlist dataset

playlist = playlist.filter(
    pl.col("name").is_in(["Me in 2023"]).not_(),
).with_columns(
    # remove extras from trackName
    pl.col("trackName").str.replace_all(r"\((.?*)\)|- (.?*)", ""),
    # extract extra descriptions from trackName column
    trackDesc=pl.col("trackName").str.extract_all(r"\((.?*)\)|- (.?*)"),
    # create custom trackId column identifier for different tracks
    trackId=pl.col("trackName").add("@").add(pl.col("artistName")),
)
playlist.sample(5)

name,lastModifiedDate,trackName,artistName,albumName,trackUri,addedDate,trackDesc,trackId
str,str,str,str,str,str,str,list[str],str
"""enything""","""2024-10-15""","""Slow Talkin'""","""Haley Heynderickx""","""Among Horses III""","""spotify:track:67QtnDkCfHCmWTwWaHN9hw""","""2024-09-29""",[],"""Slow Talkin'@Haley Heynderickx"""
"""enything""","""2024-10-15""","""New York City""","""Adrianne Lenker""","""b-sides""","""spotify:track:5McZ3LHgxlz8Fpea1oo8Ad""","""2024-09-29""",[],"""New York City@Adrianne Lenker"""
"""imported""","""2024-09-29""","""Mystery of Love""","""Sufjan Stevens""","""Call Me By Your Name (Original Motion Pi…","""spotify:track:4HbeGjbt7u3pvwDk1vN7P0""","""2023-07-05""",[],"""Mystery of Love@Sufjan Stevens"""
"""Theme Music""","""2024-11-13""","""Gullak Theme""","""Anurag Saikia""","""Gullak: Season 1 (Music from the Tvf Ori…","""spotify:track:1gRNd0m1e3VqF2OGSp5YuQ""","""2024-11-13""",[],"""Gullak Theme@Anurag Saikia"""
"""Wcowin""","""2024-04-16""","""Letter""","""iris""","""YOUR FRIENDSHIP MEANS SO MUCH TO ME""","""spotify:track:6UTEi5a5qIYkHQYNptqC0l""","""2024-04-05""",[],"""Letter@iris"""


In [28]:
(
    playlist.group_by("name")
    .agg(
        pl.col("lastModifiedDate").first().str.to_date().dt.strftime("%d %b, %y"),
        pl.len().alias("trackCount"),
        pl.col("artistName").n_unique().alias("artistCount"),
        pl.col("trackName", "artistName").unique().sample(3, with_replacement=True),
        pl.col("addedDate").str.to_date().min(),
    )
    .with_columns(
        pl.col("trackName", "artistName").list.join(" | "),
    )
    .sort("addedDate")
    .drop("addedDate")
    .style.tab_header(
        title="Playlists' Summary",
        subtitle="No. of tracks in each playlist (sorted by creation date)",
    )
)

GT(_tbl_data=shape: (20, 6)
┌─────────────┬──────────────────┬────────────┬─────────────┬───────────────────┬──────────────────┐
│ name        ┆ lastModifiedDate ┆ trackCount ┆ artistCount ┆ trackName         ┆ artistName       │
│ ---         ┆ ---              ┆ ---        ┆ ---         ┆ ---               ┆ ---              │
│ str         ┆ str              ┆ u32        ┆ u32         ┆ str               ┆ str              │
╞═════════════╪══════════════════╪════════════╪═════════════╪═══════════════════╪══════════════════╡
│ HNDi        ┆ 26 Oct, 24       ┆ 43         ┆ 30          ┆ Piyu Bole | Tumhe ┆ Mohammed Irfan | │
│             ┆                  ┆            ┆             ┆ Apna Banane       ┆ Zain Zohaib |    │
│             ┆                  ┆            ┆             ┆ Ka-Chand C…       ┆ Amit Triv…       │
│ English Hai ┆ 01 Nov, 24       ┆ 88         ┆ 71          ┆ Falling Up | a    ┆ Dan + Shay |     │
│             ┆                  ┆            ┆             ┆ thousand years |  ┆ Zedd | Henry     │
│             ┆                  ┆            ┆             ┆ Girls Li…         ┆ Moodie           │
│ Jd          ┆ 22 Jun, 23       ┆ 32         ┆ 26          ┆ Bones | Stay |    ┆ The Weeknd |     │
│             ┆                  ┆            ┆             ┆ Bara Bara' Bere   ┆ Jennifer Lopez | │
│             ┆                  ┆            ┆             ┆ Bere'             ┆ CNCO             │
│ Thend       ┆ 04 Nov, 23       ┆ 21         ┆ 6           ┆ One Right Now  |  ┆ The Weeknd |     │
│             ┆                  ┆            ┆             ┆ Take My Breath  | ┆ Swedish House    │
│             ┆                  ┆            ┆             ┆ Pray …            ┆ Mafia | The W…   │
│ Part-EE     ┆ 16 Mar, 24       ┆ 44         ┆ 39          ┆ Sade Dil Vich |   ┆ Majbul Khan |    │
│             ┆                  ┆            ┆             ┆ Are Rafta Rafta   ┆ Ram Sampath |    │
│             ┆                  ┆            ┆             ┆ Dekho | …         ┆ Tanishk Bagc…    │
│ …           ┆ …                ┆ …          ┆ …           ┆ …                 ┆ …                │
│ hnything    ┆ 30 Sep, 24       ┆ 98         ┆ 89          ┆ Hisaar Mein |     ┆ Pratsofficial |  │
│             ┆                  ┆            ┆             ┆ Aarti | Bindya    ┆ Bawari Basanti | │
│             ┆                  ┆            ┆             ┆                   ┆ Raghu D…         │
│ Wcowin      ┆ 16 Apr, 24       ┆ 62         ┆ 51          ┆ Ticking | Glow Me ┆ E.S. Posthumus | │
│             ┆                  ┆            ┆             ┆ | My Soul         ┆ Thomas Bergersen │
│             ┆                  ┆            ┆             ┆                   ┆ | July           │
│ enything    ┆ 15 Oct, 24       ┆ 92         ┆ 41          ┆ The Day I Drove   ┆ Tiny Ruins |     │
│             ┆                  ┆            ┆             ┆ the Car Around    ┆ Mark Kozelek |   │
│             ┆                  ┆            ┆             ┆ the Block…        ┆ Shantanu Pan…    │
│ anything    ┆ 02 Oct, 24       ┆ 41         ┆ 23          ┆ Watercolour |     ┆ Gintaara |       │
│             ┆                  ┆            ┆             ┆ Maybe Baby | Now  ┆ Karshni | Rachel │
│             ┆                  ┆            ┆             ┆ She Knows         ┆ Singh            │
│ Theme Music ┆ 13 Nov, 24       ┆ 50         ┆ 29          ┆ Itne Pyaar Se  |  ┆ Dream Note |     │
│             ┆                  ┆            ┆             ┆ Akash's Love      ┆ Kavita Seth |    │
│             ┆                  ┆            ┆             ┆ Theme | Po…       ┆ Simran Hora      │
└─────────────┴──────────────────┴────────────┴─────────────┴───────────────────┴──────────────────┘, _body=<great_tables._gt_data.Body object at 0x10facc510>, _boxhead=Boxhead([ColInfo(var='name', type=<ColInfoTypeEnum.default: 1>, column_label='name', column_align='left', column_width=None), ColInfo(var='lastModifiedDate', type=<Col

In [29]:
(
    playlist.group_by("name")
    .agg(
        pl.col("trackId").filter(pl.col("trackId").is_duplicated()).unique(),
    )
    .filter(pl.col("trackId").list.len().gt(0))
    .style.tab_header("Playlist containing duplicate Tracks")
)

GT(_tbl_data=shape: (3, 2)
┌─────────────────┬────────────────────────────────────────┐
│ name            ┆ trackId                                │
│ ---             ┆ ---                                    │
│ str             ┆ list[str]                              │
╞═════════════════╪════════════════════════════════════════╡
│ think exception ┆ ["Yad Lagla@Ajay Gogavale"]            │
│ English Hai     ┆ ["At My Worst@Pink Sweat$"]            │
│ Compose It !!   ┆ ["Cold Little Heart@Michael Kiwanuka"] │
└─────────────────┴────────────────────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x109596690>, _boxhead=Boxhead([ColInfo(var='name', type=<ColInfoTypeEnum.default: 1>, column_label='name', column_align='left', column_width=None), ColInfo(var='trackId', type=<ColInfoTypeEnum.default: 1>, column_label='trackId', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10facd1d0>, _spanners=Spanners([]), _heading=Heading(title='Playlist containing duplicate Tracks', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x10fc4d550>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, catego

In [30]:
(
    playlist.group_by("trackId")
    .agg(pl.col("name"))
    .filter(pl.col("name").list.len().gt(2))
    .style.tab_header("Tracks in Multiple Playlist")
)

GT(_tbl_data=shape: (0, 2)
┌─────────┬───────────┐
│ trackId ┆ name      │
│ ---     ┆ ---       │
│ str     ┆ list[str] │
╞═════════╪═══════════╡
└─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x10fc5bd50>, _boxhead=Boxhead([ColInfo(var='trackId', type=<ColInfoTypeEnum.default: 1>, column_label='trackId', column_align='left', column_width=None), ColInfo(var='name', type=<ColInfoTypeEnum.default: 1>, column_label='name', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10fc4ed90>, _spanners=Spanners([]), _heading=Heading(title='Tracks in Multiple Playlist', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x10facdb10>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=OptionsInfo(scss=True, category='heading', type='value', value='initial'), heading_subtitle_font_size=OptionsInfo(scss=True, category='heading', type='px', value='85%

## Import saved albums data to compare with playlists


In [31]:
album = (
    pl.read_json("spotify-data/YourLibrary.json")
    .select("albums")
    .explode("albums")
    .unnest("albums")
    .rename({"album": "name"})
    .rename(lambda x: "album" + x.title())
)
print(album.shape)
album.head()

(76, 3)


albumArtist,albumName,albumUri
str,str,str
"""Tommee Profitt""","""In The End""","""spotify:album:4ogDiddFmcxnT4PC9lVEoV"""
"""Various Artists""","""What Are the Odds?""","""spotify:album:33iMwGCgwBkVoi3UZgcfws"""
"""Deepak Peace""","""1947 Se Ak-47 Tak""","""spotify:album:4ovgqD1QkgAIrN3ZsuKmzG"""
"""Sharon Van Etten""","""Are We There""","""spotify:album:6EQWzHbd3EYO8J4EAIJst4"""
"""Agnes Obel""","""Myopia""","""spotify:album:1XFhwj2xUtypgyEqAmTrQV"""


In [37]:
pl_album = playlist.join(album, on="albumName")
print(pl_album.shape)
pl_album.head()

(61, 11)


name,lastModifiedDate,trackName,artistName,albumName,trackUri,addedDate,trackDesc,trackId,albumArtist,albumUri
str,str,str,str,str,str,str,list[str],str,str,str
"""imported""","""2024-09-29""","""invisible string""","""Taylor Swift""","""folklore""","""spotify:track:6VsvKPJ4xjVNKpI8VVZ3SV""","""2023-07-05""",[],"""invisible string@Taylor Swift""","""Taylor Swift""","""spotify:album:2fenSS68JI1h4Fo296JfGr"""
"""imported""","""2024-09-29""","""Mystery of Love""","""Sufjan Stevens""","""Call Me By Your Name (Original Motion Pi…","""spotify:track:4HbeGjbt7u3pvwDk1vN7P0""","""2023-07-05""",[],"""Mystery of Love@Sufjan Stevens""","""Various Artists""","""spotify:album:7K0x1O9gqMQlDwbMkyCCIM"""
"""imported""","""2024-09-29""","""hoax""","""Taylor Swift""","""folklore""","""spotify:track:0YeDG5HnKnG7jpArkzsSPa""","""2023-07-05""",[],"""hoax@Taylor Swift""","""Taylor Swift""","""spotify:album:2fenSS68JI1h4Fo296JfGr"""
"""imported""","""2024-09-29""","""Madhubala""","""Amit Trivedi""","""Songs of Love""","""spotify:track:3hcwpXiJtR7kwDrWllcH0v""","""2024-04-16""",[],"""Madhubala@Amit Trivedi""","""Amit Trivedi""","""spotify:album:0jpXgPEyyujKElDwbCWL7o"""
"""imported""","""2024-09-29""","""Teri Ay""","""Umer Farooq""","""Patang""","""spotify:track:3WBsWNtL054HCFz7UUGK9e""","""2024-04-16""",[],"""Teri Ay@Umer Farooq""","""Umer Farooq""","""spotify:album:2dtBK0Vkm1vrkucjfaqR3I"""


In [39]:
pl_album.select("name", "trackId", "albumName").style

name,trackId,albumName
imported,invisible string@Taylor Swift,folklore
imported,Mystery of Love@Sufjan Stevens,Call Me By Your Name (Original Motion Picture Soundtrack)
imported,hoax@Taylor Swift,folklore
imported,Madhubala@Amit Trivedi,Songs of Love
imported,Teri Ay@Umer Farooq,Patang
imported,Tu Aisa Kaise Hai@Osho Jain,Saar
anything,Flesh & Bone@Anoushka Maskey,Things I Saw in a Dream
anything,Hollow - acoustic@Prateek Kuhad,The Way That Lovers Do
anything,Trampoline@Anoushka Maskey,Things I Saw in a Dream
anything,Things I Saw in a Dream@Anoushka Maskey,Things I Saw in a Dream


In [18]:
artist = (
    pl.read_json("spotify-data/YourLibrary.json")
    .select("artists")
    .explode("artists")
    .unnest("artists")
    .rename(lambda x: "artist" + x.title())
)
print(artist.shape)
artist.head()

(21, 2)


artistName,artistUri
str,str
"""AMAN""","""spotify:artist:2fbOTJal9uKzOkYEWxaiCK"""
"""Adrianne Lenker""","""spotify:artist:4aKWmkWAKviFlyvHYPTNQY"""
"""Agnes Obel""","""spotify:artist:1rKrEdI6GKirxWHxIUPYms"""
"""Anuv Jain""","""spotify:artist:4gdMJYnopf2nEUcanAwstx"""
"""Bayaan""","""spotify:artist:3atMq790wQ7IqjeSO0HFeP"""


In [44]:
playlist.join(artist, on="artistName").select("name", "trackName", "artistName").style

name,trackName,artistName
imported,Mystery of Love,Sufjan Stevens
imported,Faasle,Kaavish
imported,100 words,Prateek Kuhad
imported,Teri Ay,Umer Farooq
imported,Shehron Ke Raaz,Prateek Kuhad
imported,Tu Aisa Kaise Hai,Osho Jain
imported,Uljhe Hue,Osho Jain
imported,Nindiya Re,Kaavish
anything,Hollow,Prateek Kuhad
anything,I Never Knew,Prateek Kuhad
